In [16]:
import pandas as pd
import numpy as np

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, udf, lit

spark = SparkSession.builder.appName('read csv files').getOrCreate()

In [18]:
csv_df=spark.read.csv("AprTopics.csv")

In [19]:
apr_topics=csv_df.toPandas()

In [20]:
apr_topics.columns=['date','topic','tweetsNum']

In [21]:
apr_topics.head()

,date,topic,tweetsNum
0,2021-04-14,1,203
1,2021-04-13,3,231
2,2021-04-27,1,158
3,2021-04-27,0,814
4,2021-04-21,3,95


In [22]:
apr_topics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 3 columns):
date         120 non-null object
topic        120 non-null object
tweetsNum    120 non-null object
dtypes: object(3)
memory usage: 2.9+ KB


In [23]:
apr_topics['date']=pd.to_datetime(apr_topics['date'])
#apr_topics['topic']=pd.to_datetime(apr_topics['topic'])
apr_topics['tweetsNum']=pd.to_numeric(apr_topics['tweetsNum'])

Transform the Data

In [24]:
df = apr_topics.pivot_table(values = 'tweetsNum', index = ['date'], columns = 'topic')
df.head()

topic,0,1,2,3
date,,,,
2021-04-01,961,207,204,190
2021-04-02,990,223,245,188
2021-04-03,1118,229,250,185
2021-04-04,1142,258,211,190
2021-04-05,910,168,190,131


Replace NaN Values

In [25]:
df.fillna(0, inplace=True)
df.sort_values(list(df.columns),inplace=True)
df = df.sort_index()


In [26]:
df.head()

topic,0,1,2,3
date,,,,
2021-04-01,961,207,204,190
2021-04-02,990,223,245,188
2021-04-03,1118,229,250,185
2021-04-04,1142,258,211,190
2021-04-05,910,168,190,131


Aggregate and clean data

In [31]:
df.iloc[:, 0:] = df.iloc[:, 0:].cumsum()
df.head()

topic,0,1,2,3
date,,,,
2021-04-01,961,207,204,190
2021-04-02,3873,844,857,378
2021-04-03,9854,2140,2209,563
2021-04-04,20046,4353,4471,753
2021-04-05,35359,7651,7833,884


In [32]:
df.to_csv('plot.csv')

In [28]:
!pip install bar_chart_race

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [29]:
import bar_chart_race as bcr
bcr.__version__

SyntaxError: invalid syntax (_make_chart.py, line 212)

In [ ]:
bcr.bar_chart_race(df = df, 
                   n_bars = 6, 
                   sort='desc',
                   title='tweets number under each topic in Apr',
                   filename = 'Apr_tweets.mp4')